In [4]:
import pandas as pd
import numpy as np
import requests

In [5]:
def get_data(token):

    res = requests.get(
        f'https://api.cryptowat.ch/markets/coinbase-pro/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )
    if res.json().get('error',0)!=0:
        res = requests.get(
            f'https://api.cryptowat.ch/markets/kraken/{token}usd/ohlc',
            params={
                'periods': '3600',
                'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
            }
        )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [6]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT', 'FLOW']

In [7]:
df_lis = [get_data(token) for token in tokens]
df = pd.concat(df_lis,axis=0)

In [8]:
df['close_ratio']=df.loc[df['token'] == 'ETH', 'close']   /  df.loc[df['token'] == 'SOL', 'close']

In [9]:
df.groupby('token')['volumeUSD'].sum().to_frame().sort_values(by=['volumeUSD'])

,volumeUSD
token,
FLOW,9.243792e+07
USDT,2.128105e+09
AVAX,3.950965e+09
SOL,7.666180e+09
ETH,2.337456e+10


In [10]:
df.rename(columns={'volumeUSD': 'volumeTerm', 'volume': 'volumeBase'}, inplace=True)

In [11]:
close=df.pivot(index='ts', columns='token', values='close')
